In [1]:
#Python
import requests
import re
import pandas as pd
import numpy as np

#nltk
import nltk
from nltk.corpus import stopwords

#BERT
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments
import torch
from bs4 import BeautifulSoup

#sklearn
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score


import requests
import re

In [2]:
comment_vs_likes = pd.read_excel('todos_con_nltk.xlsx')

In [23]:
comment_vs_likes

,Comment,Likes,sentiment_nltk
0,luzu parecia porta jajajajajajajaaj,0,0.0000
1,velada del ano 4 jagger vs ippo,0,0.0000
2,en el de viruz y momo que honda el casco de mo...,0,0.0000
3,lo de luzu y lolito vi la pelea muy inestables,0,0.0000
4,tremendo evento pero por lo menos en la parte ...,0,-0.5423
...,...,...,...
26388,hjajajaj,0,0.0000
26389,vamooo,1,0.0000
26390,tercero,0,0.0000
26391,segundo,1,0.0000


# Digo que si es mayor a 20 likes [predict] =1

In [24]:
comment_vs_likes['predict'] = (comment_vs_likes['Likes'] >= 20).astype(int)

# Me quedo con todos los comentarios que son 1 y con 6000 que son 0

In [43]:
n = len(comment_vs_likes[comment_vs_likes['predict'] == 0])
print(n)
# Seleccionar aleatoriamente las filas a eliminar
drop_indices = np.random.choice(comment_vs_likes[comment_vs_likes['predict'] == 0].index, n - 8000, replace=False)

# Eliminar las filas seleccionadas
comment_vs_likess = comment_vs_likes.drop(drop_indices)

#Eliminar valores nulos en comments
comment_vs_likess.dropna(subset=['Comment'], inplace=True)

#Reseteamos la llave
comment_vs_likess = comment_vs_likess.reset_index(drop=True)

#Eliminamos stopwords
stop_words = set(stopwords.words('spanish'))
comment_vs_likess['Comment'] = comment_vs_likess['Comment'].apply(lambda x: ' '.join([word for word in str(x).split() if word not in (stop_words)]))

#Nueva columna para generar un nuevo sentimiento con BERT

tokenizer = AutoTokenizer.from_pretrained('nlptown/bert-base-multilingual-uncased-sentiment')

model = AutoModelForSequenceClassification.from_pretrained('nlptown/bert-base-multilingual-uncased-sentiment')

def sentiment_score(review):
    tokens = tokenizer.encode(review, return_tensors='pt')
    result = model(tokens)
    return int(torch.argmax(result.logits))+1

comment_vs_likess['sentiment_BERT'] = comment_vs_likess['Comment'].apply(lambda x: sentiment_score(str(x)[:512]))

25577


# Clasificador usando BERT

In [6]:
comments_vs_likess = pd.read_excel('sentimientos8kBERT.xlsx')

In [5]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased')

C:\Users\Mario\anaconda3\lib\site-packages\huggingface_hub\file_download.py:133: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Mario\.cache\huggingface\hub. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


In [9]:
comments_vs_likess

,Comment,Likes,sentiment_nltk,predict,sentiment_BERT
0,velada ano 4 jagger vs ippo,0,0.0000,0,1
1,merecia carola ganar,0,0.0000,0,5
2,quiero ser sentimental aca abuela viruz,1,0.0258,0,1
3,espectacular ibai logrado evento aplausos verd...,0,0.3400,0,5
4,reunion personas menos 72 iq,0,0.0000,0,3
...,...,...,...,...,...
8719,combate bueno,39,0.0000,1,4
8720,siiiuuuu,0,0.0000,0,3
8721,tongo,0,0.0000,0,3
8722,vamooo,1,0.0000,0,1


In [44]:
def process_data(row):
    #Seleciono la fila comment
    text = row['Comment']
    #la convierto a string
    text = str(text)
    #Se eliminan los espacios adicionales en el texto y luego el split divide por comas cada palabra del comentario
    text = ' '.join(text.split())
    #Se tokeniza con las tres capas: encoding, attention y selfattention de un transformer
    encodings = tokenizer(text, padding="max_length", truncation=True, max_length=128)
    
    
    #Para sentimiento lo dejo tal cual
    #label = 0
    #if row['sentiment_BERT'] == 'positive':
    #    label += 1
                                    ##############################
                                    #                            #         
                                    # Aqui se ponen las columnas #
                                    #                            #
                                    ##############################
    
    #encodings['label'] = row['sentiment_BERT']
    #encodings['label_nltk'] = row['sentiment_nltk']
    encodings['text'] = text
    encodings['predict'] = row['predict']

    return encodings

In [19]:
print(process_data({
    'Comment': 'this is a sample review of a movie. omgo mgom og ',
    'sentiment_BERT': '1',
    'sentiment_nltk':'0.5',
    'predict':0
}))

{'input_ids': [101, 2023, 2003, 1037, 7099, 3319, 1997, 1037, 3185, 1012, 18168, 3995, 11460, 5358, 13958, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

In [45]:
#Aqui eliges las columnas que quieres que entre, tambien entra aqui la variable a predecir
df=comments_vs_likess[['Comment','predict']]

#Aqui se procesa todas esas columnas que queremos que entre a nuestro modelo BERT
processed_data = []

for i in range(len(df[:1000])):
    processed_data.append(process_data(df.iloc[i]))

In [46]:
from sklearn.model_selection import train_test_split

new_df = pd.DataFrame(processed_data)

train_df, valid_df = train_test_split(
    new_df,
    test_size=0.2,
    random_state=2022
)

In [47]:
import pyarrow as pa
from datasets import Dataset

train_hg = Dataset(pa.Table.from_pandas(train_df))
valid_hg = Dataset(pa.Table.from_pandas(valid_df))

In [48]:
from transformers import AutoModelForSequenceClassification

#Aqui se le dice que nuestro modelo de clasificacion tiene 4 columnas
model = AutoModelForSequenceClassification.from_pretrained(
    'bert-base-uncased',
    num_labels=2
)

loading configuration file config.json from cache at C:\Users\Mario/.cache\huggingface\hub\models--bert-base-uncased\snapshots\0a6aa9128b6194f4f3c4db429b6cb4891cdb421b\config.json
Model config BertConfig {
  "_name_or_path": "bert-base-uncased",
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.24.0",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 30522
}

loading weights file pytorch_model.bin from cache at C:\Users\Mario/.cache\huggingface\hub\models--bert-base-uncased\snapshots\0a6aa9128b6194f4f3c4db429b6cb489

In [73]:
def compute_loss(model, inputs, return_outputs=False):
    outputs = model(**inputs)
    loss = outputs.loss
    return (loss, outputs) if return_outputs else loss

In [82]:
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    return {"accuracy": (predictions.argmax(axis=-1) == labels).mean()}

In [78]:
from transformers import DataCollatorForLanguageModeling

data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer, 
    mlm=True, 
    mlm_probability=0.15
)

In [85]:
from transformers import AdamW, TrainingArguments, Trainer

training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=3,
    per_device_train_batch_size=2048,
    per_device_eval_batch_size=2048,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=10,
    load_best_model_at_end=True,
    evaluation_strategy='steps',
    eval_steps=500,
    learning_rate=5e-5
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_hg,
    eval_dataset=valid_hg,
    data_collator=data_collator,
    compute_metrics=compute_metrics
)


PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


In [86]:
#Se entrena
trainer.train()

The following columns in the training set don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: __index_level_0__, predict, text. If __index_level_0__, predict, text are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running training *****
  Num examples = 800
  Num Epochs = 3
  Instantaneous batch size per device = 2048
  Total train batch size (w. parallel, distributed & accumulation) = 2048
  Gradient Accumulation steps = 1
  Total optimization steps = 3
  Number of trainable parameters = 109483778


RuntimeError: [enforce fail at C:\b\abs_bao0hdcrdh\croot\pytorch_1675190257512\work\c10\core\impl\alloc_cpu.cpp:81] data. DefaultCPUAllocator: not enough memory: you tried to allocate 629145600 bytes.